In [6]:
from pydantic import BaseModel
from pydantic_ai import Agent
from pydantic_ai.models.ollama import OllamaModel

# Define the Ollama model
ollama_model = OllamaModel(
    model_name="llama3.2",  # Replace with your preferred model
    base_url="http://ollama:11434/v1/"  # Ollama's default base URL
)


class UserProfile(BaseModel):
    user_id: str
    topics: dict[str, float]  # Dictionary of topics with weights
    average_ctr: float
    session_length: int


# Create the agent
agent = Agent(model=ollama_model, result_type=UserProfile)


In [7]:
async def generate_user_profile():
    prompt = """
    Create a synthetic user profile. Include:
    - user_id: A unique identifier (e.g., 'synthetic_001').
    - topics: A dictionary of topic preferences with weights (e.g., {'space': 0.6, 'science': 0.4}).
    - average_ctr: The average click-through rate (e.g., 0.75).
    - session_length: The average number of clicks per session (e.g., 5).
    Format the response as JSON.
    """
    result = await agent.run(prompt)
    print(result.data)  # Validated UserProfile object
    print(result.cost())  # Token usage details

In [12]:
generate_user_profile()

<coroutine object generate_user_profile at 0x7fd10a6ae730>